### BUFN400 Fall 2023---University of Maryland 

# Homework 5: Portfolio Optimization and Predicting Returns with OLS Regressions

### By James Zhang 

### November 27, 2023

#### There are probably some typos in in this homework since this is the first time I have used this particular problem set.  Please send me an email if you find typos!!


In [139]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.linear_model import LinearRegression
warnings.filterwarnings("ignore")
sp_df = pd.read_csv("sp500.csv")
treasury_df = pd.read_csv("treasury.csv")

# Problem 0

(This problem is almost the same as for Homework 1. You are welcome to cut and paste from that homework.)

1. Under "CRSP" / "Annual Update" / "Index / Treasury and Inflation", download "US Treasury and Inflation Indexes". Choose the entire time period, monthly data, and "Both" (return and level), all variables. Show the number of rows and columns, the names of the columns, and the first few rows of data.

In [140]:
print(f"Number of rows: {treasury_df.shape[0]}, and number of columns: {treasury_df.shape[1]}")
print(f"The columns names are {[column for column in treasury_df.columns]}")
treasury_df.head()

Number of rows: 1165, and number of columns: 21
The columns names are ['caldt', 'b30ret', 'b30ind', 'b20ret', 'b20ind', 'b10ret', 'b10ind', 'b7ret', 'b7ind', 'b5ret', 'b5ind', 'b2ret', 'b2ind', 'b1ret', 'b1ind', 't90ret', 't90ind', 't30ret', 't30ind', 'cpiret', 'cpiind']


,caldt,b30ret,b30ind,b20ret,b20ind,b10ret,b10ind,b7ret,b7ind,b5ret,...,b2ret,b2ind,b1ret,b1ind,t90ret,t90ind,t30ret,t30ind,cpiret,cpiind
0,1925-12-31,NaN,50.27088,NaN,49.26875,NaN,46.97544,NaN,40.4141,NaN,...,NaN,39.54598,NaN,39.84357,NaN,34.83693,NaN,38.91082,NaN,42.1
1,1926-01-30,NaN,50.27088,NaN,49.26875,NaN,46.97544,NaN,40.4141,NaN,...,NaN,39.54598,NaN,39.84357,0.004042,34.97774,0.002951,39.02565,0.000000,42.1
2,1926-02-27,NaN,50.27088,NaN,49.26875,NaN,46.97544,NaN,40.4141,NaN,...,NaN,39.54598,NaN,39.84357,0.003237,35.09096,0.002768,39.13367,0.000000,42.1
3,1926-03-31,NaN,50.27088,NaN,49.26875,NaN,46.97544,NaN,40.4141,NaN,...,NaN,39.54598,NaN,39.84357,0.002778,35.18843,0.002778,39.24238,-0.005587,41.9
4,1926-04-30,NaN,50.27088,NaN,49.26875,NaN,46.97544,NaN,40.4141,NaN,...,NaN,39.54598,NaN,39.84357,0.003072,35.29653,0.003072,39.36293,0.005618,42.1


2. Under "CRSP" / "Annual Update" / Index / S&P 500 Indexes", download "CRSP Index File on the S&P 500". Choose the entire time period, monthly data, and all variables.  Show the number of rows and columns, the names of the columns, and the first few rows of data.

In [141]:
print(f"Number of rows: {sp_df.shape[0]}, and number of columns: {sp_df.shape[1]}")
print(f"The columns names are {[column for column in sp_df.columns]}")
sp_df.head()

Number of rows: 1165, and number of columns: 11
The columns names are ['caldt', 'vwretd', 'vwretx', 'ewretd', 'ewretx', 'totval', 'totcnt', 'usdval', 'usdcnt', 'spindx', 'sprtrn']


,caldt,vwretd,vwretx,ewretd,ewretx,totval,totcnt,usdval,usdcnt,spindx,sprtrn
0,1925-12-31,NaN,NaN,NaN,NaN,15236829.5,89,NaN,NaN,12.46,NaN
1,1926-01-30,-0.001783,-0.003980,0.006457,0.003250,15277664.0,89,15236829.5,79.0,12.74,0.022472
2,1926-02-27,-0.033296,-0.037876,-0.039979,-0.042451,14712894.9,89,15277664.0,81.0,12.18,-0.043956
3,1926-03-31,-0.057708,-0.062007,-0.067915,-0.073275,14012079.2,89,14712894.9,81.0,11.46,-0.059113
4,1926-04-30,0.038522,0.034856,0.031441,0.027121,14500482.2,89,14012079.2,82.0,11.72,0.022688


3. Merge the two monthly datasets by date into one dataset. Show the number of rows and columns, the names of the columns, and the first few rows of data. (Hint: If the dates do not match exactlly in the merge, you might merge by YYYMM instead of YYYMMDD.)

* Hint: To merge data, you need a key (date). An example of merging data was covered in the Python bootcamp. When unexpeted issues  arise, the options `how='outer'` and `indicator=True` can be helpful for diagnosing problems. To make dates line up correctly, consider using `pd.tseries.offsets.MonthEnd(...)`.


In [142]:
# Convert 'caldt' to datetime in both dataframes
sp_df['caldt'] = pd.to_datetime(sp_df['caldt']) + pd.tseries.offsets.MonthBegin()
treasury_df['caldt'] = pd.to_datetime(treasury_df['caldt']) + pd.tseries.offsets.MonthBegin()
df = pd.merge(sp_df, treasury_df, on='caldt', how='outer')
print(f"Number of rows: {df.shape[0]}, and number of columns: {df.shape[1]}")
print(f"The columns names are {[column for column in df.columns]}")
df.head()

Number of rows: 1165, and number of columns: 31
The columns names are ['caldt', 'vwretd', 'vwretx', 'ewretd', 'ewretx', 'totval', 'totcnt', 'usdval', 'usdcnt', 'spindx', 'sprtrn', 'b30ret', 'b30ind', 'b20ret', 'b20ind', 'b10ret', 'b10ind', 'b7ret', 'b7ind', 'b5ret', 'b5ind', 'b2ret', 'b2ind', 'b1ret', 'b1ind', 't90ret', 't90ind', 't30ret', 't30ind', 'cpiret', 'cpiind']


,caldt,vwretd,vwretx,ewretd,ewretx,totval,totcnt,usdval,usdcnt,spindx,...,b2ret,b2ind,b1ret,b1ind,t90ret,t90ind,t30ret,t30ind,cpiret,cpiind
0,1926-01-01,NaN,NaN,NaN,NaN,15236829.5,89,NaN,NaN,12.46,...,NaN,39.54598,NaN,39.84357,NaN,34.83693,NaN,38.91082,NaN,42.1
1,1926-02-01,-0.001783,-0.003980,0.006457,0.003250,15277664.0,89,15236829.5,79.0,12.74,...,NaN,39.54598,NaN,39.84357,0.004042,34.97774,0.002951,39.02565,0.000000,42.1
2,1926-03-01,-0.033296,-0.037876,-0.039979,-0.042451,14712894.9,89,15277664.0,81.0,12.18,...,NaN,39.54598,NaN,39.84357,0.003237,35.09096,0.002768,39.13367,0.000000,42.1
3,1926-04-01,-0.057708,-0.062007,-0.067915,-0.073275,14012079.2,89,14712894.9,81.0,11.46,...,NaN,39.54598,NaN,39.84357,0.002778,35.18843,0.002778,39.24238,-0.005587,41.9
4,1926-05-01,0.038522,0.034856,0.031441,0.027121,14500482.2,89,14012079.2,82.0,11.72,...,NaN,39.54598,NaN,39.84357,0.003072,35.29653,0.003072,39.36293,0.005618,42.1


In [143]:
df["vwretx"].mean() * 12

0.07564811340206185

# Problem 1

Consider the problem of forming an optimal portfolio consisting of various sets of assets. Suppose the investor has log utility. Use the sample means, variances, and covariances to form estimates of the "true" means and variance. Under the assumption that the sample means, variances, and covariance are the best estimates of the "true" means, variances, and covariances, calculate optimal portfolio weights for the various assets as a fraction of invested wealth. Also calculate the "certainty equivalent" value of the opportunity to invest in these assets.  Perform the calculations for for the following sets of assets:

### Solution

1. T-bills and the value-weighted SP500.

Here, we have treasury bills, a riskless asset, and the value-weighted SP500, which is a risky asset. We have previously proves in class that log utility investors choose the growth-optimal portfolio, which maximizes internal rate of return, in this situation.
$$x = \frac{\pi}{\sigma^2}$$

In [144]:
def growth_optimal_portfolio(risk_free_asset: pd.Series, risky_asset: pd.Series):
    """
    Helper function that calculates growth optimal allocation in the risky asset and 
    certainty equivalent
    """
    # Calculate risk free, risk premium, annual variance
    rf = risk_free_asset.mean() * 12
    pi = (risky_asset - risk_free_asset).mean() * 12
    annual_variance = risky_asset.var() * 12
    x = pi / annual_variance
    certainty_equivalent = rf + x * pi
    return x, certainty_equivalent

In [187]:
"""
1. T-bills and value-weighted SP500
"""
x, certainty_equivalent = growth_optimal_portfolio(df["t30ret"], df["vwretx"])
print(f"Optimal weights: equal={round(x, 4)}, T-bills={round(1-x, 4)}")
print(f"Certainty equivalent = {certainty_equivalent}")

Optimal weights: equal=1.2399, T-bills=-0.2399
Certainty equivalent = 0.08614995372424422


2. T-bills and equally-weighted SP500.

This example is very similar to the one above.

In [188]:
"""
2. T-bills and equally-weighted SP500
"""
(x), certainty_equivalent = growth_optimal_portfolio(df["t30ret"], df["ewretx"])
print(f"Optimal weights: equal={round(x, 4)}, T-bills={round(1-x, 4)}")
print(f"Certainty equivalent = {certainty_equivalent}")

Optimal weights: equal=1.2817, T-bills=-0.2817
Certainty equivalent = 0.1208987525797573


3. T-bills, the value-weighted SP500, and the equally-weighted SP500.



In [147]:
def two_risky_assets(risk_free: pd.Series, asset: pd.Series, market: pd.Series):
    rf = risk_free.mean() * 12
    # Let asset be some asset and let market be the "market index"
    # Drop nans in case we don't have data for either of the assets
    df = pd.DataFrame({"asset": asset, "market": market}).dropna()
    asset, market = df["asset"], df["market"]
    # Finding alpha, beta, and variances
    r_a, r_m = asset.mean() * 12, market.mean() * 12
    cov_matrix = np.cov(asset, market)
    beta_a = cov_matrix[0][1] / cov_matrix[1][1]
    alpha_a = r_a - rf - beta_a * (r_m - rf)
    var_a, var_m = asset.var() * 12, market.var() * 12
    # Finding optimal weights
    x_a = alpha_a / var_a
    pi = (market - risk_free).mean() * 12
    x_m = (pi / var_m) - beta_a * x_a    
    certainty_equivalent = rf + x_a * (alpha_a + beta_a * pi) + x_m * pi
    return (x_a, x_m, - 1 * (x_a + x_m - 1)), certainty_equivalent

In [189]:
"""
3. T-bills, the value-weighted SP500, and the equally-weighted SP500.
"""
weights, certainty_equivalent = two_risky_assets(df["t30ret"], df["vwretx"], df["ewretx"])
(x_a, x_m, x_f) = weights
print(f"Note that we are not leveraged: {sum(weights)}")
# Printing out weights and expected portfolio returns
print(f"Optimal weights: value-weighted SP={round(x_a, 4)}, equal-weighted SP={round(x_m, 4)}, T-bills={round(x_f, 4)}")
print(f"Certainty equivalent = {certainty_equivalent}")

Note that we are not leveraged: 1.0
Optimal weights: value-weighted SP=-0.2788, equal-weighted SP=1.497, T-bills=-0.2181
Certainty equivalent = 0.1236435067848684


4. T-bills, the value-weighted SP500, and UST 10-year notes.

In [190]:
"""
4. T-bills, the value-weighted SP500, and the UST 10-year notes
"""
weights, certainty_equivalent = two_risky_assets(df["t30ret"], df["b10ret"], df["vwretx"])
(x_a, x_m, x_f) = weights
print(f"Note that we are not leveraged: {sum(weights)}")
# Printing out weights and expected portfolio returns
print(f"Optimal weights: 10Y Note={round(x_a, 4)}, Value-Weighted SP500={round(x_m, 4)}, T-bills={round(x_f, 4)}")
print(f"Certainty equivalent = {certainty_equivalent}")

Note that we are not leveraged: 1.0
Optimal weights: 10Y Note=3.4264, Value-Weighted SP500=2.1902, T-bills=-4.6166
Certainty equivalent = 0.20297786344217783


5. T-bills, the value-weighted SP500, the equally-weighted SP500, and UST 10-year notes.

In [166]:
def fsolve_svd(pi, sigma, rtol=1e-15, num_polish=1):
    u, s, vt = np.linalg.svd(sigma, full_matrices=False, hermitian=True)
    sinv = np.where(s > rtol, 1.00 / s, 0.00)
    x = vt.T @ (sinv * (u.T @ pi))
    # Polish solution:
    for _ in range(num_polish):
        x = x + vt.T @ (sinv * (u.T @ (pi - sigma @ x)))
    return x

def many_risky_assets(risk_free, assets: pd.DataFrame):
    N = len(assets)
    R, rf = assets.mean() * 12, risk_free.mean() * 12
    pi = R - rf
    sigma = np.cov(assets.dropna().T)
    x = fsolve_svd(pi, sigma, rtol=1e-14, num_polish=2)
    certainty_equivalent = rf + np.transpose(R - rf) @ x
    return x, certainty_equivalent

In [192]:
"""
4. T-bills, the value-weighted SP500, equally-weighted SP500 and the UST 10-year notes
"""
rf = df["t30ret"].mean() * 12
weights, certainty_equivalent = many_risky_assets(df["t30ret"], df[["vwretx", "ewretx", "b10ret"]])
weights = list(weights)
weights.insert(0, 1 - sum(weights))
print(f"Note that we are not leveraged: {sum(weights)}")
# Printing out weights and expected portfolio returns
print(f"Optimal weights: T-bills={weights[0]}, value={weights[1]}, equal={weights[2]}, 10Y note={weights[3]}")
print(f"Certainty equivalent = {certainty_equivalent + weights[0] * rf}")

Note that we are not leveraged: 0.9999999999999929
Optimal weights: T-bills=-58.521054177093454, value=-68.73849690737207, equal=84.36570747085986, 10Y note=43.89384361360566
Certainty equivalent = 1.8281816915007791


# Problem 2

1. Consider a linear regression of the excess return of the equally-weighted index on the excess return of the value-weighted index using the 30-day T-bill return as the risk-free rate: 
$$
R_t^{\text{ewretd}} - R_t^{\text{t30}} = \alpha + \beta \cdot (R_t^{\text{vwretd}} - R_t^{\text{t30}}) + \epsilon .
$$
For a training dataset consisting of the first half of the data, calculate estimates of $\alpha$ and $\beta$ using OLS. Also calculate estimates of the means and variances of the two excess returns and the variance of the residuals. Construct a scatter plot of the two excess returns and show the fitted regression line on the same plot.  


2. Calculate the approximately optimal portfolio and the Sharpe ratio for a hypothetical log-utility investor who uses the estimated regression coefficients, means, and variances as if they were the correct theoretical coefficients and variances.  Explain why the Sharpe ratio is likely to be a biased estimate of more realistic portfolio performance.


3. Plot the realized Sharpe ratio and realized internal rate of return achieved by various portfolio investing fraction $x$ in the equally-weighted index and $1-x$ in the value-weighted index.

Now consider how to use the second half of the data to test the model.  Predicting future returns is difficult. Specifically, the estimated mean return on the two indexes are subject to error because the return standard deviations each month are large relative to the monthly means. To apply the model to the test data, use for a predicted value of $\alpha$ some fraction of the estimate of $\alpha$ from the OLS regression. An reasonable but simplistic ad hoc value might be one-half the estimated $\alpha$. Using one-half of the estimated $\alpha$ is a simplistic, ad hoc way to approximate a Bayesian adjustment based on an initial belief that the unknown true $\alpha$ has an expected value equal to of zero and the variance of this initial estimate is the same as the variance of the difference between the true $\alpha$ and its OLS estimate. More generally, instead of using weights of one half, one might use weights inversely proportional to the variance of the errors of the prior estimate (of zero) and the variance of the error from the OLS estimate.

Instead of also deflating the risk premium on the value-weighted index by a factor of one half, it would be appropriate not to deflate it at all if our initial expectation was that the risk premium was likely to be different from zero and probably positive. 


4. Using this "conservative" estimate of $\alpha$ (deflated by one half), calculate the optimal portfolio for the log-utility investor, plot the returns on this portfolio, and calculate the achieved Sharpe ratio.


5. Also calculate OLS estimates for the test dataset. Provide an intuitive discussion of whether deflating the estimates of $\alpha$ by one-half was a good idea.


6. (Extra Credit) Implement the methodology for this problem in a more sophisticated manner by using for any month a training dataset consisting of all previous months.

Hints: Write the OLS regression as a function which can be called more than once. Let the beginning and ending dates used in the regression be function arguments. Also let the ad hoc Bayesian parameter (of one half) be a function argument.

# Problem 3

This problem will consider how to use market information to forecast returns on a market index.

Define three "signals" (dividend yield, past returns, volatility) which might be used to forecast returns on three market indexes (value-weighted, equally-weighted, and SP500):

1. Construct the past year's dividend yield on each index as a monthly time series starting one year after the beginning of the data. (Hint: Each month's dividend yield can be approximated from the difference between returns with and without dividends; a simple approximation to the dividend yield is the sum of these differences.)

2. Construct a series of past long-term returns over the previous ten years. (The return over the previous ten years is the amount one dollar invested in the index ten years previously would be worth at the current date.) Define the signal as the log of this past ten-year return. Explain why taking the log might be a sensible empirical strategy for this problem.

3. Use the daily dataset to construct an estimate for each month volatility by using the same standard deviation of the previous month's daily log returns.

4. Now let $x_t$ denote one of the three signals constructed in 1, 2, 3 above, and let $r_t$ denote the excess return on one of the three indexes. Let $f(x)$ denote some transformation of the data; here we will use $f(x) = \frac{x - \bar{x}}{\sigma_x}$, where $\bar{x}$ is the sample mean of $x_t$ and $\sigma_x$ is the sample standard deviation of $x_t$. Consider the predictive regression 
$$
r_t = \alpha + \beta \cdot f(x_t) + \epsilon_t
$$
As before, divide the entire time period into a training period and testing period of equal length, calculate OLS estimates for $\alpha$ and $\beta$.

5. Calculate an optimal portfolio under the assumptions that the estimated parameters are correct versions of the true parameters. Plot the returns on this portfolio against the returns on the index itself for the training period. Obviously, your market-timing model will beat a buy and hold strategy in the training period it was estimated on. (Note: Since there are three index portfolios and three signals, you might present 9 plots in a 3 x 3 grid if you have time.)

6. Now apply the model out of sample by dampening the estimate of $\beta$ by some ad hoc factor, such as one half. Calculate the optimal portfolio for a log utility investor. Plot the returns achieved out of sample for the estimated optimal portfolio. Calculate the Sharpe ratio for the optimal portfolio. Plot the Sharpe ratio achieved in the test period when the investor holds a fraction $x$ of wealth in the optimal portfolio and a fraction $1-x$ in the (untimed) index. 

7. Tell a brief economic, financial, or behavioral story from why you obtain the results above.

8. (Extra Credit) Generalize using past ten-year returns to using past returns over various different periods from one day to ten years. Do you find momentum over some periods and reversals over other periods.

